# ETL EDA users_items

In [2]:
from ast import literal_eval
import datetime as dt
import pandas as pd
import fastparquet
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', None)

In [4]:
df_users_items = pd.read_parquet("archivos_parquet/df_users_items.parquet")

In [ ]:
# pd.set_option('display.max_colwidth', None)
df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [5]:
# Descompone la lista en filas separadas
df_users_items_final = df_users_items.explode('items')

In [6]:
# Resetear índece del dataframe
df_users_items_final.reset_index(drop=True, inplace=True)

In [7]:
# Creamos las nuevas columnas para cada clave
df_users_items_final = pd.concat([df_users_items_final, pd.json_normalize(df_users_items_final['items'])], axis=1)

In [8]:
# eliminar columnas
df_users_items_final.drop(["items_count", "steam_id", "items", "playtime_2weeks"], axis=1, inplace=True)
df_users_items_final.head()

,user_id,user_url,item_id,item_name,playtime_forever
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0


In [ ]:
# función para obtener datos varios de los dataframe(info, describe, null, duplicated)
def print_df_info(df):

    print("** Info - Información general **")
    print(df.info())

    print("=======================================================")

    print("** Shape **")
    print(f"Filas: {df.shape[0]} - Columnas: {df.shape[1]}")

    print("=======================================================")

    print("** Describe - Estadísticas resumidas **")
    print(df.describe().T)

    print("=======================================================")

    print("** NaN - Nulos **")
    print(df.isna().sum())

    print("=======================================================")
    print("** Duplicated - Duplicados **")
    print(df.duplicated().sum())

In [ ]:
# print_df_info(df_users_items_2)
print_df_info(df_users_items_final)

** Info - Información general **
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   user_url          object 
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(1), object(4)
memory usage: 197.2+ MB
None
** Shape **
Filas: 5170015 - Columnas: 5
** Describe - Estadísticas resumidas **
                      count        mean          std  min  25%   50%    75%  \
playtime_forever  5153209.0  991.495141  5418.204405  0.0  0.0  34.0  355.0   

                       max  
playtime_forever  642773.0  
** NaN - Nulos **
user_id                 0
user_url                0
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64
** Duplicated - Duplicados **
59209


In [ ]:
# ver dupicados, no son duplicados del todo, los item_id son diferentes, lo que sugiere mismo
# usuario juegos diferentes
df_users_items_final.loc[df_users_items_final.duplicated()]

,user_id,user_url,item_id,item_name,playtime_forever
75762,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,NaN,NaN,NaN
164518,Nikiad,http://steamcommunity.com/id/Nikiad,20,Team Fortress Classic,5.0
164519,Nikiad,http://steamcommunity.com/id/Nikiad,50,Half-Life: Opposing Force,0.0
164520,Nikiad,http://steamcommunity.com/id/Nikiad,70,Half-Life,0.0
164521,Nikiad,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0.0
...,...,...,...,...,...
4910939,76561198080057659,http://steamcommunity.com/profiles/76561198080...,221910,The Stanley Parable,53.0
4910940,76561198080057659,http://steamcommunity.com/profiles/76561198080...,261030,The Walking Dead: Season Two,253.0
4910941,76561198080057659,http://steamcommunity.com/profiles/76561198080...,273110,Counter-Strike Nexon: Zombies,0.0
4910942,76561198080057659,http://steamcommunity.com/profiles/76561198080...,730,Counter-Strike: Global Offensive,0.0


In [ ]:
# ver nulos en item_id
df_users_items_final[df_users_items_final['item_id'].isnull()]

,user_id,user_url,item_id,item_name,playtime_forever
3733,Wackky,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN
3849,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN
6019,hellom8o,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN
6523,starkillershadow553,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN
7237,darkenkane,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN
...,...,...,...,...,...
5169470,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN
5169471,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN
5169472,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN


In [9]:
# eliminar nulos de item_id, no proporcionan información del juego
df_users_items_final.dropna(subset=['item_id'], inplace = True)

In [ ]:
# verifico la eliminación
df_users_items_2[df_users_items_2['item_id'].isnull()]

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever


In [10]:
# ver la dimensión final despues de la elimminación
df_users_items_final.shape

(5153209, 5)

In [11]:
# eliminar duplicados
df_users_items_final.drop_duplicates(inplace=True)
df_users_items_final.shape

(5094092, 5)

In [12]:
# para reducir los datos, elimino los que tengan 0 en playtime_forever
df_users_items_final = df_users_items_final[df_users_items_final['playtime_forever'] != 0]
df_users_items_final.shape

(3246362, 5)

In [15]:
df_users_items_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3246362 entries, 0 to 5170013
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   user_url          object 
 2   item_id           int64  
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 148.6+ MB


In [14]:
# cambiar el tipo de dato de item_id
df_users_items_final['item_id'] = df_users_items_final['item_id'].astype(int)

In [17]:
# reset del índice
df_users_items_final.reset_index(drop=True, inplace=True)

In [18]:
# verifico los datos
df_users_items_final.head()

,user_id,user_url,item_id,item_name,playtime_forever
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,300,Day of Defeat: Source,4733.0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,240,Counter-Strike: Source,1853.0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3830,Psychonauts,333.0


In [19]:
# exportar el dataset
df_users_items_2 = df_users_items_final.to_parquet("datasets/df_users_items_2.parquet")